# Step 1: Setup & Imports

In [12]:
### Imports ###
import matplotlib.pyplot as plt 
import numpy as np
import random 
import math
import pandas as pd
import csv
import os
import sys
import json
PATH_TO_DIRECTORY = os.getcwd()
PATH_TO_DATA_DIRECTORY = os.getcwd() + "/data/"

In [13]:
### Utility Functions ###
def coeffGen(p,k):
    """
    Input: Lower limit and higher limit (int)
    Output: List of length K with random values between low and high
    """
    coeffSeq = []
    for i in range(k):
        # mean = a, var = b^2
        coeffSeq.append(np.random.normal(0, 1.0/np.power(i+1, p)))
    return coeffSeq

def genTrigFun(a_k, b_k):
    """
    Input: 
    >> int list a_k, b_k: Sequences of length K
    
    Output: 
    >> fun f(t): Desired trig function with coefficients corresponding to sequences a_k, b_k
    """
    def fun_t (t):
        sum = 0
        for i in range(len(a_k)):
            sum += a_k[i]*np.sin(i*t) + b_k[i]*np.cos(i*t)
        return sum
    return (fun_t)

def euclidDist(a, b):
    return np.sqrt(np.power(a, 2) + np.power(b, 2))

def speed(x_t, y_t):
    """
    Input: 2 arrays of length of TIME_T
    Output: 1 array of length of TIME_T representing the speed 
    """
    return euclidDist(np.gradient(x_t), np.gradient(y_t))
    
def curvature(x_t, y_t):
    """
    Input: 2 arrays of length of TIME_T
    Output: 1 array of length of TIME_T representing the curvature
    """    
    num = abs(np.gradient(x_t)*np.gradient(np.gradient(y_t))
             - np.gradient(y_t)*np.gradient(np.gradient(x_t)))
    denom = np.power(speed(x_t, y_t), 3)
    return (num/denom)

def changeResolution (num, res):
    if num == 0 or res == 0 or math.isnan(num) or math.isnan(res):
        return 0
    return math.floor(num * res) / res

# Step 2: Generating Curves

In [14]:
### Constants ###
NUM_CURVES = 50
ZFILL_LEN = int(np.ceil(np.log10(NUM_CURVES)))
NUM_TERMS = 50
POWER_OF_TERMS = 2.1
SIZE = 100

In [15]:
### Generating Curve Functions ###
def genCurve(k, p, size, file_no):
    """
    Inputs: 
    >> int k: Number of terms in trig series 
    >> int p: Power of denominator when sampled from normal distribution
    >> int s: size - boundary box size
    >> int file_no: Index, used for naming files
    
    Outputs:
    >> Dataframe with columns [a_k, b_k, c_k, d_k], coefficients used to generate parametric curve
    
    Writes the data to Curves/coefficients_(file_no) in a csv file.
    """
    
    # Coefficients - 1 x K
    a_k = coeffGen(p, k)
    b_k = coeffGen(p, k)
    c_k = coeffGen(p, k)
    d_k = coeffGen(p, k)
    
    # Theta
    theta = random.random()**0.5 * (2*math.pi)
    
    x_min = size/2.0*random.random()**2
    x_max = size - size/2.0*random.random()**2
    
    y_min = size/2.0*random.random()**2
    y_max = size - size/2.0*random.random()**2
    
    # Transform data into dataframe
    data = np.transpose(np.array((a_k, b_k, c_k, d_k)))
    df1 = pd.DataFrame(data, columns = ['a_k', 'b_k', 'c_k', 'd_k'])
    extras = np.transpose(np.array([theta, size, x_min, x_max, y_min, y_max]))
    df2 = pd.DataFrame(extras, columns = ['extras'])
    df = pd.concat([df1,df2], axis=1)
    
    # Export csv
    curve_file_csv = PATH_TO_DATA_DIRECTORY + "curve_data/coefficients_{}.csv".format(str(file_no).zfill(ZFILL_LEN))
    df.to_csv(curve_file_csv, index=False)
    return df

def genNCurves(n, k, p, s):
    """
    Inputs: 
    >> int n: Number of curves we want to generate
    >> int k: Number of terms in trig series 
    >> int p: Power of denominator when sampled from normal distribution
    
    Outputs nothing.
    Writes n csv files with desired curve data.
    """
    for i in range(n):
        # Uniformly sample k from a range of 2 to 100 to get a bigger variety of curves
        k = random.randrange(2,100)
        # Use grid size 100x100
        genCurve(k, p, 100, i)

In [16]:
genNCurves(NUM_CURVES, NUM_TERMS, POWER_OF_TERMS, SIZE)

# Step 3: Capturing Curves

In [17]:
### Capture Functions ###
def captureCurve(coeff_path, time_start, time_stop, frame_rate, resolution, plot=False):
    """
    Inputs: 
    >> str coeff_path:                 Path to coefficients_xx.csv
    >> float time_start, time_stop:   Beginning and end times to generate time step increments
    >> int frame_rate:                How often to sample the time step between time_start and time_stop
    >> string file_out:               Name of output file 
    >> bool plot:                     Plots the captured curves if set to true
    
    Outputs:
    >> dataframe df:                  Dataframe with columns [X, Y] 
    """    

    # Read in data from coefficients csv
    # Coefficients - 1 x K
    data = pd.read_csv(coeff_path)
    
    a_k = data['a_k'].values
    b_k = data['b_k'].values
    c_k = data['c_k'].values
    d_k = data['d_k'].values
    
    theta = data['extras'][0]
    
    size = data['extras'][1]
    
    x_min = data['extras'][2]
    x_max = data['extras'][3]
    x_dif = x_max - x_min
    
    y_min = data['extras'][4]
    y_max = data['extras'][5]
    y_dif = y_max - y_min
    
    
    time_t = np.arange(time_start, time_stop, 1/frame_rate)
    p_time_t = theta * (time_t - time_start) / (time_stop - time_start)

    x_fun = genTrigFun(a_k, b_k)
    y_fun = genTrigFun(c_k, d_k)
    
    # Get minimum and maximum x,y coordinates of the graph 
    
    x_og = x_fun(p_time_t) # 1 x len(TIME_T)
    y_og = y_fun(p_time_t) # 1 x len(TIME_T)
    
    lower_xlim = min(x_og)
    upper_xlim = max(x_og)
    
    lower_ylim = min(y_og)
    upper_ylim = max(y_og)
    
    # Coordinates - 1 x len(TIME_T)
    
    x_enlarged = []
    y_enlarged = []
    for i in range(0, len(time_t)):
        x_enlarged.append((x_dif / (upper_xlim - lower_xlim) * (x_og[i] - lower_xlim) + x_min))
        y_enlarged.append((y_dif / (upper_ylim - lower_ylim) * (y_og[i] - lower_ylim) + y_min))
    
    # Transform coordinates based on resolution
    
    x = []
    y = []
    for i in range(0, len(time_t)):
        x.append(np.array(changeResolution(x_enlarged[i], resolution)))
        y.append(np.array(changeResolution(y_enlarged[i], resolution)))

    if(plot):
        # Plots
        plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.5, hspace=0.5)
        plt.subplot(121)
        plt.plot(x, y)
        plt.title("Curve no. " + str(file_no) + ", m = " + str(len(a_k)))
        plt.axis([0,size,0,size])

        plt.subplot(122)
        kSeq = np.arange(0, len(a_k), 1)
        plt.plot(kSeq, a_k)
        plt.plot(kSeq, b_k)
        plt.plot(kSeq, c_k)
        plt.plot(kSeq, d_k)
        plt.title("Coefficients")
        plt.axis([0,20,-1,1])

        plt.show()

    # Transform data into dataframe
    data = np.transpose(np.array((x, y)))
    df = pd.DataFrame(data, columns = ['X', 'Y'])
    
    summaryStats = df.describe()
    return df, summaryStats

# Step 4: Data Generation for Robustness Tests

In [18]:
### Robustness Test Constants ###
DEFAULT_FR = 24
DEFAULT_RES = 2
DEFAULT_START = 0
DEFAULT_STOP = 1

In [19]:
### Frame Rate Tests 1 (Increasing toward 24 FPS) ###
TEST_FR = [6, 9, 12, 15, 18, 21, 24]
for curve_no in range(NUM_CURVES):
    curve_str = str(curve_no).zfill(ZFILL_LEN)
    jsonItems = []
    coeff_path = PATH_TO_DATA_DIRECTORY + "curve_data/coefficients_{}.csv".format(curve_str)
    resolution = DEFAULT_RES
    for frame_rate in TEST_FR:
        fr_str = str(frame_rate).zfill(2)
        # Generate Capture Data
        df, _ = captureCurve(coeff_path, DEFAULT_START*60, DEFAULT_STOP*60, frame_rate, resolution, plot=False)
        # Save Capture Data to CSV
        dirName = PATH_TO_DATA_DIRECTORY + 'FR_test1_data'
        try:
            # Create target Directory
            os.mkdir(dirName)
        except FileExistsError:
            pass
        csv_filename = dirName + "/CRV_{}_FR_{}.dat".format(curve_str, fr_str)
        df.to_csv(csv_filename)
        # Save Capture Data to JSON
        jsonItem = {
            "name": "FR_TEST1_CRV_{}_FR_{}".format(curve_str, fr_str),
            "data_file_location": csv_filename,
            "animal_attributes":
            {
                "species": "Magic Scoliosis Fish",
                "exp_type": "MCS funtimes",
                "ID": curve_str,
                "control_group": "False"
            },
            "capture_attributes":
            {
                "dim_x": 100,
                "dim_y": 100,
                "pixels_per_mm": resolution,
                "frames_per_sec": frame_rate,
                "start_time": DEFAULT_START,
                "end_time": DEFAULT_STOP,
                "baseline_start_time": DEFAULT_START,
                "baseline_end_time": DEFAULT_STOP
            }
        }
        jsonItems.append(jsonItem)
    dirName = PATH_TO_DATA_DIRECTORY + 'FR_json1_data'
    try:
        # Create target Directory
        os.mkdir(dirName)
    except FileExistsError:
        pass
    outfilename = dirName + "/CRV_{}.json".format(curve_str)
    jsonstr = json.dumps(jsonItems, indent=4)
    with open(outfilename, "w") as outfile:
        outfile.write(jsonstr)
    print("Wrote the information into %s" % outfilename)

Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json1_data/CRV_00.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json1_data/CRV_01.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json1_data/CRV_02.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json1_data/CRV_03.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json1_data/CRV_04.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json1_data/CRV_05.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json1_data/CRV_06.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json1_data/CRV_07.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json1_data/CRV_08.json
W

In [20]:
### Frame Rate Tests 2 (Increasing away from 24 FPS) ###
TEST_FR = [24, 36, 48, 60, 72, 84, 96]
for curve_no in range(NUM_CURVES):
    curve_str = str(curve_no).zfill(ZFILL_LEN)
    jsonItems = []
    coeff_path = PATH_TO_DATA_DIRECTORY + "curve_data/coefficients_{}.csv".format(curve_str)
    resolution = DEFAULT_RES
    for frame_rate in TEST_FR:
        fr_str = str(frame_rate).zfill(2)
        # Generate Capture Data
        df, _ = captureCurve(coeff_path, DEFAULT_START*60, DEFAULT_STOP*60, frame_rate, resolution, plot=False)
        # Save Capture Data to CSV
        dirName = PATH_TO_DATA_DIRECTORY + 'FR_test2_data'
        try:
            # Create target Directory
            os.mkdir(dirName)
        except FileExistsError:
            pass
        csv_filename = dirName + "/CRV_{}_FR_{}.dat".format(curve_str, fr_str)
        df.to_csv(csv_filename)
        # Save Capture Data to JSON
        jsonItem = {
            "name": "FR_TEST2_CRV_{}_FR_{}".format(curve_str, fr_str),
            "data_file_location": csv_filename,
            "animal_attributes":
            {
                "species": "Magic Scoliosis Fish",
                "exp_type": "MCS funtimes",
                "ID": curve_str,
                "control_group": "False"
            },
            "capture_attributes":
            {
                "dim_x": 100,
                "dim_y": 100,
                "pixels_per_mm": resolution,
                "frames_per_sec": frame_rate,
                "start_time": DEFAULT_START,
                "end_time": DEFAULT_STOP,
                "baseline_start_time": DEFAULT_START,
                "baseline_end_time": DEFAULT_STOP
            }
        }
        jsonItems.append(jsonItem)
    dirName = PATH_TO_DATA_DIRECTORY + 'FR_json2_data'
    try:
        # Create target Directory
        os.mkdir(dirName)
    except FileExistsError:
        pass
    outfilename = dirName + "/CRV_{}.json".format(curve_str)
    jsonstr = json.dumps(jsonItems, indent=4)
    with open(outfilename, "w") as outfile:
        outfile.write(jsonstr)
    print("Wrote the information into %s" % outfilename)

Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json2_data/CRV_00.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json2_data/CRV_01.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json2_data/CRV_02.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json2_data/CRV_03.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json2_data/CRV_04.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json2_data/CRV_05.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json2_data/CRV_06.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json2_data/CRV_07.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/FR_json2_data/CRV_08.json
W

In [21]:
### Resolution Tests 1 (Increasing Towards 2) ###
TEST_RES = [0.5, 0.75, 1, 1.25, 1.5, 1.75, 2]
for curve_no in range(NUM_CURVES):
    curve_str = str(curve_no).zfill(ZFILL_LEN)
    jsonItems = []
    coeff_path = PATH_TO_DATA_DIRECTORY + "curve_data/coefficients_{}.csv".format(curve_str)
    frame_rate = DEFAULT_FR
    for resolution in TEST_RES:
        res_str = str(resolution).zfill(2)
        # Generate Capture Data
        df, _ = captureCurve(coeff_path, DEFAULT_START*60, DEFAULT_STOP*60, frame_rate, resolution, plot=False)
        # Save Capture Data to CSV
        dirName = PATH_TO_DATA_DIRECTORY + 'RES_test1_data'
        try:
            # Create target Directory
            os.mkdir(dirName)
        except FileExistsError:
            pass
        csv_filename = dirName + "/CRV_{}_RES_{}.dat".format(curve_str, fr_str)
        df.to_csv(csv_filename)
        # Save Capture Data to JSON
        jsonItem = {
            "name": "RES_TEST1_CRV_{}_RES_{}".format(curve_str, res_str),
            "data_file_location": csv_filename,
            "animal_attributes":
            {
                "species": "Magic Scoliosis Fish",
                "exp_type": "MCS funtimes",
                "ID": curve_str,
                "control_group": "False"
            },
            "capture_attributes":
            {
                "dim_x": 100,
                "dim_y": 100,
                "pixels_per_mm": resolution,
                "frames_per_sec": frame_rate,
                "start_time": 0,
                "end_time": 60,
                "baseline_start_time": 0,
                "baseline_end_time": 60
            }
        }
        jsonItems.append(jsonItem)
    dirName = PATH_TO_DATA_DIRECTORY + 'RES_json1_data'
    try:
        # Create target Directory
        os.mkdir(dirName)
    except FileExistsError:
        pass
    outfilename = dirName + "/CRV_{}.json".format(curve_str)
    jsonstr = json.dumps(jsonItems, indent=4)
    with open(outfilename, "w") as outfile:
        outfile.write(jsonstr)
    print("Wrote the information into %s" % outfilename)

Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json1_data/CRV_00.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json1_data/CRV_01.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json1_data/CRV_02.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json1_data/CRV_03.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json1_data/CRV_04.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json1_data/CRV_05.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json1_data/CRV_06.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json1_data/CRV_07.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json1_data/CRV_

In [22]:
### Resolution Tests 2 (Increasing away from 2) ###
TEST_RES = [2, 3, 4, 5, 6, 7, 8]
for curve_no in range(NUM_CURVES):
    curve_str = str(curve_no).zfill(ZFILL_LEN)
    jsonItems = []
    coeff_path = PATH_TO_DATA_DIRECTORY + "curve_data/coefficients_{}.csv".format(curve_str)
    frame_rate = DEFAULT_FR
    for resolution in TEST_RES:
        res_str = str(resolution).zfill(2)
        # Generate Capture Data
        df, _ = captureCurve(coeff_path, DEFAULT_START*60, DEFAULT_STOP*60, frame_rate, resolution, plot=False)
        # Save Capture Data to CSV
        dirName = PATH_TO_DATA_DIRECTORY + 'RES_test2_data'
        try:
            # Create target Directory
            os.mkdir(dirName)
        except FileExistsError:
            pass
        csv_filename = dirName + "/CRV_{}_RES_{}.dat".format(curve_str, fr_str)
        df.to_csv(csv_filename)
        # Save Capture Data to JSON
        jsonItem = {
            "name": "RES_TEST2_CRV_{}_RES_{}".format(curve_str, res_str),
            "data_file_location": csv_filename,
            "animal_attributes":
            {
                "species": "Magic Scoliosis Fish",
                "exp_type": "MCS funtimes",
                "ID": curve_str,
                "control_group": "False"
            },
            "capture_attributes":
            {
                "dim_x": 100,
                "dim_y": 100,
                "pixels_per_mm": resolution,
                "frames_per_sec": frame_rate,
                "start_time": 0,
                "end_time": 60,
                "baseline_start_time": 0,
                "baseline_end_time": 60
            }
        }
        jsonItems.append(jsonItem)
    dirName = PATH_TO_DATA_DIRECTORY + 'RES_json2_data'
    try:
        # Create target Directory
        os.mkdir(dirName)
    except FileExistsError:
        pass
    outfilename = dirName + "/CRV_{}.json".format(curve_str)
    jsonstr = json.dumps(jsonItems, indent=4)
    with open(outfilename, "w") as outfile:
        outfile.write(jsonstr)
    print("Wrote the information into %s" % outfilename)

Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json2_data/CRV_00.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json2_data/CRV_01.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json2_data/CRV_02.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json2_data/CRV_03.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json2_data/CRV_04.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json2_data/CRV_05.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json2_data/CRV_06.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json2_data/CRV_07.json
Wrote the information into /Users/elainewijaya/locomotion_resources/fishmapping/data/RES_json2_data/CRV_